In [6]:
import pandas as pd
import numpy as np

df=pd.read_csv("D:\\230968364\\wkke5-main\\weatherAUS.csv")
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142188,2017-06-20,Uluru,3.5,21.8,0.0,NaN,NaN,E,31.0,ESE,...,27.0,1024.7,1021.2,NaN,NaN,9.4,20.9,No,0.0,No
142189,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,...,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,0.0,No
142190,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,...,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,0.0,No
142191,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,...,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,0.0,No


In [7]:
df.isnull().sum()

Date                 0
Location             0
MinTemp            637
MaxTemp            322
Rainfall          1406
Evaporation      60843
Sunshine         67816
WindGustDir       9330
WindGustSpeed     9270
WindDir9am       10013
WindDir3pm        3778
WindSpeed9am      1348
WindSpeed3pm      2630
Humidity9am       1774
Humidity3pm       3610
Pressure9am      14014
Pressure3pm      13981
Cloud9am         53657
Cloud3pm         57094
Temp9am            904
Temp3pm           2726
RainToday         1406
RISK_MM              0
RainTomorrow         0
dtype: int64

### Preprocess the data, remove the attributes which were are not useful to predict rain. Also, remove rows with at least one missing value for each of them.

In [8]:
columns_to_remove = ['Date', 'Location', 'WindGustDir', 'WindDir9am', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm']
df.drop(columns=columns_to_remove, inplace=True)
df.dropna(inplace=True)
df['RainToday'] = df['RainToday'].apply(lambda x: 1 if x == 'Yes' else 0)
df['RainTomorrow'] = df['RainTomorrow'].apply(lambda x: 1 if x == 'Yes' else 0)
df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,RainToday,RISK_MM,RainTomorrow
5939,17.9,35.2,0.0,12.0,12.3,48.0,SW,6.0,20.0,20.0,13.0,1006.3,1004.4,0,0.0,0
5940,18.4,28.9,0.0,14.8,13.0,37.0,SSE,19.0,19.0,30.0,8.0,1012.9,1012.1,0,0.0,0
5942,19.4,37.6,0.0,10.8,10.6,46.0,NNW,30.0,15.0,42.0,22.0,1012.3,1009.2,0,0.0,0
5943,21.9,38.4,0.0,11.4,12.2,31.0,WSW,6.0,6.0,37.0,22.0,1012.7,1009.1,0,0.0,0
5944,24.2,41.0,0.0,11.2,8.4,35.0,WNW,17.0,13.0,19.0,15.0,1010.7,1007.4,0,0.0,0


### Calculate the Variance Inflation Factor (VIF) value. VIF is a number that determines whether a variable has multicollinearity or not (starts from 1, and it has no upper limit. If the number gets larger, it means the variable has huge multicollinearity on it.).

In [9]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
df_numeric = df.select_dtypes(include=[np.number])
df_numeric.dropna(inplace=True)
X = add_constant(df_numeric)
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

         Variable           VIF
0           const  38274.189068
1         MinTemp      5.259314
2         MaxTemp      7.471926
3        Rainfall      1.539822
4     Evaporation      2.184574
5        Sunshine      2.059200
6   WindGustSpeed      2.953963
7    WindSpeed9am      1.816001
8    WindSpeed3pm      2.087723
9     Humidity9am      2.536509
10    Humidity3pm      3.903886
11    Pressure9am     18.569891
12    Pressure3pm     18.704391
13      RainToday      1.759800
14        RISK_MM      1.435861
15   RainTomorrow      1.760936


### Remove multicollinearities by creating new features. Find the features that have paired values and create the new feature which is the difference value between those pairs.

In [10]:
corr_matrix = df.corr()
highly_corr_pairs = []
threshold = 0.9
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:
            highly_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j]))
for col1, col2 in highly_corr_pairs:
    new_feature = col1 + "_minus_" + col2
    df[new_feature] = df[col1] - df[col2]
    df.drop([col1, col2], axis=1, inplace=True)

df

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,RainToday,RISK_MM,RainTomorrow,Pressure3pm_minus_Pressure9am
5939,17.9,35.2,0.0,12.0,12.3,48.0,SW,6.0,20.0,20.0,13.0,0,0.0,0,-1.9
5940,18.4,28.9,0.0,14.8,13.0,37.0,SSE,19.0,19.0,30.0,8.0,0,0.0,0,-0.8
5942,19.4,37.6,0.0,10.8,10.6,46.0,NNW,30.0,15.0,42.0,22.0,0,0.0,0,-3.1
5943,21.9,38.4,0.0,11.4,12.2,31.0,WSW,6.0,6.0,37.0,22.0,0,0.0,0,-3.6
5944,24.2,41.0,0.0,11.2,8.4,35.0,WNW,17.0,13.0,19.0,15.0,0,0.0,0,-3.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139108,19.3,33.4,0.0,6.0,11.0,35.0,NE,9.0,20.0,63.0,32.0,0,0.0,0,-3.4
139109,21.2,32.6,0.0,7.6,8.6,37.0,SE,13.0,11.0,56.0,28.0,0,0.0,0,-3.4
139110,20.7,32.8,0.0,5.6,11.0,33.0,W,17.0,11.0,46.0,23.0,0,0.0,0,-3.5
139111,19.5,31.8,0.0,6.2,10.6,26.0,NNW,9.0,17.0,62.0,58.0,0,0.0,0,-4.2


### Remove features that have a VIF value above 5.

In [11]:
df_numeric = df.select_dtypes(include=[np.number])
df_numeric.dropna(inplace=True)
X = add_constant(df_numeric)
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

vif_data = calculate_vif(X)
print("Initial VIF values:")
print(vif_data)
while vif_data["VIF"].max() > 5:
    feature_to_remove = vif_data.sort_values("VIF", ascending=False)["Variable"].iloc[0]
    print(f"Removing feature: {feature_to_remove} with VIF: {vif_data['VIF'].max()}")
    X = X.drop(columns=[feature_to_remove])
    vif_data = calculate_vif(X)
print("\nFinal VIF values after removing high VIF features:")
print(vif_data)

Initial VIF values:
                         Variable         VIF
0                           const  121.529194
1                         MinTemp    5.207614
2                         MaxTemp    7.289786
3                        Rainfall    1.538033
4                     Evaporation    2.184474
5                        Sunshine    2.055324
6                   WindGustSpeed    2.697293
7                    WindSpeed9am    1.800622
8                    WindSpeed3pm    2.087679
9                     Humidity9am    2.533784
10                    Humidity3pm    3.896439
11                      RainToday    1.750438
12                        RISK_MM    1.435652
13                   RainTomorrow    1.728211
14  Pressure3pm_minus_Pressure9am    1.416859
Removing feature: const with VIF: 121.52919414158517
Removing feature: MaxTemp with VIF: 60.19161344409008
Removing feature: WindGustSpeed with VIF: 23.33109076102826
Removing feature: Humidity9am with VIF: 21.932849378584915
Removing feature: 

### Build a regression model to perform the Rain prediction. Also, tabulate accuracy of the prediction models, before and VIF computation.

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

df_numeric = df.select_dtypes(include=[np.number])
df_numeric.dropna(inplace=True)
X = df_numeric.drop(columns=['RainTomorrow'])
y = df_numeric['RainTomorrow']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred_initial = model.predict(X_test)
mae_initial = mean_absolute_error(y_test, y_pred_initial)
r2_initial = r2_score(y_test, y_pred_initial)
print(f"Initial Mean Absolute Error (MAE): {mae_initial}")
print(f"Initial R-squared (R2): {r2_initial}")
X_with_constant = add_constant(X)  
vif_data = pd.DataFrame()
vif_data["Variable"] = X_with_constant.columns
vif_data["VIF"] = [variance_inflation_factor(X_with_constant.values, i) for i in range(X_with_constant.shape[1])]
while vif_data["VIF"].max() > 5:
    feature_to_remove = vif_data.sort_values("VIF", ascending=False)["Variable"].iloc[1]  # Exclude the constant
    print(f"Removing feature: {feature_to_remove} with VIF: {vif_data['VIF'].max()}")
    X_with_constant = X_with_constant.drop(columns=[feature_to_remove])
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X_with_constant.columns
    vif_data["VIF"] = [variance_inflation_factor(X_with_constant.values, i) for i in range(X_with_constant.shape[1])]
X_train_new = X_train[X_with_constant.columns[1:]]  
X_test_new = X_test[X_with_constant.columns[1:]]
model_new = LinearRegression()
model_new.fit(X_train_new, y_train)
y_pred_after_vif = model_new.predict(X_test_new)
mae_after_vif = mean_absolute_error(y_test, y_pred_after_vif)
r2_after_vif = r2_score(y_test, y_pred_after_vif)
results = {
    "Metric": ["MAE", "R-squared (R2)"],
    "Before VIF Removal": [mae_initial, r2_initial],
    "After VIF Removal": [mae_after_vif, r2_after_vif]
}

results_df = pd.DataFrame(results)
print("\nModel Comparison (Before and After VIF Removal):")
print(results_df)

Initial Mean Absolute Error (MAE): 0.22654729024187828
Initial R-squared (R2): 0.41680688398744814
Removing feature: MaxTemp with VIF: 120.30543916808108
Removing feature: Humidity3pm with VIF: 79.36207353735423
Removing feature: WindGustSpeed with VIF: 72.56923517506866
Removing feature: Evaporation with VIF: 62.59247908138429
Removing feature: RainToday with VIF: 57.25296185644229
Removing feature: Humidity9am with VIF: 57.12335461643993
Removing feature: WindSpeed3pm with VIF: 14.70813897607397
Removing feature: RISK_MM with VIF: 12.377337839709577
Removing feature: Sunshine with VIF: 12.256298707176422
Removing feature: Pressure3pm_minus_Pressure9am with VIF: 9.275359186552356
Removing feature: MinTemp with VIF: 7.307057453664427

Model Comparison (Before and After VIF Removal):
           Metric  Before VIF Removal  After VIF Removal
0             MAE            0.226547           0.321942
1  R-squared (R2)            0.416807           0.062565
